<a href="https://colab.research.google.com/github/floriandendorfer/demand-estimation/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. PACKAGES

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import IV2SLS
import matplotlib.pyplot as plt
from scipy import optimize
import seaborn as sns
from scipy.stats import norm



---



## 2. DATA

In [ ]:
#!rm -rf demand-estimation
!git clone https://github.com/floriandendorfer/demand-estimation.git
data = pd.read_csv('demand-estimation/data1.csv',index_col=0)



---



## 3. FIRST LOOK AT THE DATA

### 3.1 VARIABLES

In [ ]:
print(data.columns)

Each row contains sales information for an ice cream brand (i.e., Häagen-Dazs, Ben \& Jerry's) and a geographic market (i.e., a county).

*   `county` is the geopgrahpic market identifier ($t$ in the slides).
*   `Häagen-Dazs` is 1 if the ice cream brand is Häagen-Dazs and 0 if the ice cream brand is Ben \& Jerry's.
*   `price` is the dollar price an ice cream serving of that brand is sold at.
*   `fuel cost` is the dollar cost of transporting an ice cream serving.
*   `sales` is the number of ice cream servings of that brand sold per week.
*   `county size` is the number of *total* ice cream servings sold per week.


### 3.2 DATA DESCRIPTION

Let's have a brief look at the data.

In [ ]:
print(data.head(20))

Pick your favorite county.

In [ ]:
t = 10

In [ ]:
print('In total,',data['county size'][t],'ice cream servings are sold in county',data['county'][t],'. As',data['sales'][t],' of them were ',np.where(data['Häagen-Dazs'][t] == 0, "Ben & Jerry's","Häagen-Dazs")," ice cream, that brand's market share in this county is ",(100*data['sales'][t]/data['county size'][t]).round(2),'%. The price of a serving of', np.where(data['Häagen-Dazs'][t] == 0, "Ben & Jerry's","Häagen-Dazs"),'ice cream in this county is $',data['price'][t].round(2),'.')

In [ ]:
data.describe()


1.   How many counties are there?
2.   What is the average price per serving?
2.   What is the average market size in the sample? What is the largest market size?
3.   What is the median number of ice cream servings sold per county for a given brand?





## 4. MARKET SHARES

### 4.1 COMPUTING MARKET SHARES

In [ ]:
data['s'] = data['sales']/data['county size']

Calculate the market share of each firm (Häagen-Dazs, Ben & Jerry's) in each county based on the number of ice cream units sold.


### 4.2 COMPARING MARKET SHARES AND PRICES

In [ ]:
data.groupby('Häagen-Dazs')[['price','s']].describe().loc[:, (slice(None), ['count', 'mean', 'std'])]

Compare Häagen-Dazs and Ben & Jerry's in terms of their market shares and prices across counties.

1.   In how many counties is Häagen-Dazs ice cream sold? Ben \& Jerry's?
2.   Which ice cream brand is more expensive? Which one has the larger market share?
3.   Which ice cream brand do you think consumers prefer?
4.   For a given brand, do market shares vary across counties? If so, why do you think that is?


---



### 4.3 EVALUATING MARKET CONCENTRATION

Häagen-Dazs and Ben \& Jerry's are the only products in the market for 'super-premium' ice cream. Let's calculate the Hirschmann-Hifendahl Index (HHI) for county $t$:
$$10000\times\frac{\sum_{j\in\{H,B\}}\text{sales}_{jt}^2}{(\sum_{j\in\{H,B\}}\text{sales}_{jt})^2}$$


In [ ]:
hhi1 = 10000*data.groupby('county')['s'].apply(lambda x: ((x/x.sum())**2).sum())

In [ ]:
print('The average county-level HHI is',hhi1.mean().astype(int),'.')

We could also report the HHI across counties:
$$ 10000\times \frac{\sum_{j\in\{H,B\}}(\sum_t \text{sales}_{jt})^2}{(\sum_{j\in\{H,B\}}\sum_t \text{sales}_{jt})^2} $$

In [ ]:
hhi2 = 10000*(data[data['Häagen-Dazs'] == 1]['sales'].sum()**2 + data[data['Häagen-Dazs'] == 0]['sales'].sum()**2)/(data[data['Häagen-Dazs'] == 1]['sales'].sum() + data[data['Häagen-Dazs'] == 0]['sales'].sum())**2


In [ ]:
print('The average HHI across counties is',hhi2.mean().astype(int),'.')

Is the market for 'super-premium' ice cream highly concentrated, moderately concentrated or unconcentrated?

### 4.4 COMPUTING THE 'OUTSIDE OPTION' MARKET SHARE

In [ ]:
data['s0'] = 1 - data.groupby(['county'])['s'].transform('sum')

Define the 'outside option' market share for each county. Here the 'outside good' is any ice cream sold other than Ben & Jerry's or Hägen-Dazs.

## 5. DEMAND ESTIMATION

### 5.1 TRANSFORMING THE DATA

Transform the market shares to back out the **mean utilities**. The transformed market share is going to be the **dependent variable** in the regressions we run. Remember:
$$ \ln(s_{jt}) - \ln(s_{0t}) = \alpha p_{jt} + \mathbf{x}_{jt}\prime\boldsymbol{ \beta } + \xi_{jt} $$

In [ ]:
Y = np.log(data['s']) - np.log(data['s0'])

In the regression, the **independent variables** are going to be the price and the Häagen-Dazs dummy variable, plus a constant.
$$ V_{jt} = \beta_0 + \beta_\text{Häagen-Dazs}\mathbf{1}(j = \text{Häagen-Dazs}) + \alpha p_{jt} + \xi_{jt} $$

In [ ]:
X=sm.add_constant(data[['price','Häagen-Dazs']])

### 5.2 OLS REGRESSION

In [ ]:
ols = sm.OLS(Y,X)
ols_result = ols.fit(cov_type='HC3')
ols_result.summary()

Let's interpret the estimation results.

1.   All else equal, how much lower is the mean utility of consuming a unit of ice cream if the unit price increases by \$1?
2.   On average, which ice cream brand do consumers prefer?
3.   What is the meaning of the constant? It is negative. Why do you think that is?
4. What is the mean utility of consuming a unit of Ben & Jerry's ice cream if it is sold for \$3?

### 5.3 ENDOGENEITY AND INSTRUMENTAL VARIABLE

Our estimate of the price coefficient is likely biased. The true coefficients are

\begin{align*}
  \beta_0 = & -0.44 \\
  \beta_{Häagen-Dazs} = & -1.07 \\
  \alpha = & -0.63
\end{align*}

Why is that? Do you think the fuel price is a good **instrumental variable** for the unit price?

In [ ]:
first_stage = sm.OLS(data['price'],sm.add_constant(data[['fuel cost','Häagen-Dazs']]))
first_stage_result = first_stage.fit(cov_type='HC3')
first_stage_result.summary()

1. Are the estimates significantly different from zero in a statistical sense?
2. According to our estimates, by how much does the unit price increase if the fuel cost increases by \$1 (and everything else is unchanged)? Does this intuitively make sense?

We plot Häagen-Dazs market shares and unit prices and fit a line (left). We compare this to the results of a 2SLS regression with fuel price as the instrument (right).

In [ ]:
figure, axis = plt.subplots(1, 2,sharex=True,sharey=True,figsize=(9,3))

axis[0].scatter(data[data['Häagen-Dazs'] == 0]['price'],data[data['Häagen-Dazs'] == 0]['s']*100)
b, a = np.polyfit(data[data['Häagen-Dazs'] == 0]['price'], data[data['Häagen-Dazs'] == 0]['s']*100, deg=1)
axis[0].plot(np.linspace(data[data['Häagen-Dazs'] == 0]['price'].min(), data[data['Häagen-Dazs'] == 0]['price'].max(), num=80), a + b * np.linspace(data[data['Häagen-Dazs'] == 0]['price'].min(), data[data['Häagen-Dazs'] == 0]['price'].max(), num=80), color="red", lw=2.5)
axis[0].set_title('W/o instrument')

axis[1].scatter(data[data['Häagen-Dazs'] == 0]['price'],data[data['Häagen-Dazs'] == 0]['s']*100)
a,b = IV2SLS(data[data['Häagen-Dazs'] == 0]['s']*100, sm.add_constant(data[data['Häagen-Dazs'] == 0]['price']), instrument = sm.add_constant(data[data['Häagen-Dazs'] == 0]['fuel cost'])).fit().params
axis[1].plot(np.linspace(data[data['Häagen-Dazs'] == 0]['price'].min(), data[data['Häagen-Dazs'] == 0]['price'].max(), num=80), a + b * np.linspace(data[data['Häagen-Dazs'] == 0]['price'].min(), data[data['Häagen-Dazs'] == 0]['price'].max(), num=80), color="red", lw=2.5)
axis[1].set_title('W/ instrument')

figure.text(0.5, 0.04, 'price (in $)', ha='center')
figure.text(0.04, 0.5, 'market share (in %)', va='center', rotation='vertical')
plt.show()

Can you explain why the line on the right is steeper?

### 5.4 2SLS REGRESSION

Next, we re-estimate demand using the fuel cost to instrument the price of ice cream.

In [ ]:
twosls = IV2SLS(np.log(data['s']) - np.log(data['s0']), sm.add_constant(data[['price','Häagen-Dazs']]), instrument = sm.add_constant(data[['fuel cost','Häagen-Dazs']]))
twosls_result = twosls.fit()
const, alpha, beta = twosls_result.params.round(2)
twosls_result.summary()

Compare the OLS and 2SLS price coefficients. Which one has a larger magnitude?

## 6. BACKING OUT $c$ AND $\xi$

### 6.1 MARGINAL COSTS

Based on our estimates, we can back out the marginal costs of Häagen-Dazs' and Ben \& Jerry's using the FOCs of their profit maximization problems. Recall:
$$ \frac{\partial \pi_{jt}(p_{jt}^*)}{\partial p_{jt}} = \alpha s_{jt}(1-s_{jt})(p_{jt}^* - c_{jt}) + s_{jt} = 0 $$
$$ \widehat c_{jt} = p_{jt}^* + \frac{1}{\widehat \alpha(1-s_{jt})} $$

In [ ]:
C_hat = data['price'] + 1/(alpha*(1-data['s']))

In [ ]:
print('The average cost of supplying one unit of Häagen-Dazs ice-cream is $', C_hat[1::2].mean().round(2),'.')
print('The average cost of supplying one unit of Ben & Jerry\'s ice-cream is $', C_hat[0::2].mean().round(2),'.')

The true marginal costs are \$0.5 and \$1 for Häagen-Dazs and Ben & Jerry's ice cream respectively.

### 6.2 UNOBSERVED DEMAND SHOCKS

Rearrange the regression equation.
$$ \widehat \xi_{jt} = \ln(s_{jt}) - \ln(s_{0t}) - (\widehat β_0+\widehat \beta_{Häagen-Dazs}\textbf{1}(j=\text{Häagen-Dazs})+\widehat \alpha p_{jt}) $$


In [ ]:
Xi_hat = Y - (const + beta*data['Häagen-Dazs'] + alpha*data['price'])

In [ ]:
sns.kdeplot(Xi_hat)
plt.xlabel(r'$\widehat \xi_{hat}$')
plt.ylabel('density')
plt.xlim(-4,+4)
plt.show()

## 7. COMPUTING A COUNTERFACTUAL EQUILIBRIUM

Suppose we want to find out how much Häagen-Dazs's profit would change if sold ice cream as good as Ben \& Jerry's (i.e., if the Häagen-Dazs coefficient was zero) assuming its marginal cost did not change.

First, we calculate the variable profit using the data.

In [ ]:
pi_0 = ((data[data['Häagen-Dazs'] == 1]['sales'])*(data[data['Häagen-Dazs'] == 1]['price'] - C_hat[1::2])).sum()
print('Häagen-Dazs\'s variable profit in the current equilibrium are $',pi_0.astype(int),'.')

To find the new prices (and market shares) we need to find the new Nash equilibrium. That is, we need to find the set of prices that satisfy firms' FOCs.

In [ ]:
def foc(P,C,Xi,params):
    const,alpha,beta = params
    return alpha*s(P,Xi,params)*(1-s(P,Xi,params))*(P-C) + s(P,Xi,params)


To calculate `foc` we have to write functions that calculate market shares (`s`) which in turn depend on the mean utilities (`V`).



In [ ]:
def s(P,Xi,params):
    return np.exp(V(P,Xi,params))/(1 + np.exp(V(P,Xi,params)).sum())

def V(P,Xi,params):
    const,alpha,beta = params
    return const + alpha*np.array(P) + beta*np.array([0,1]) + Xi

Let's use our 2SLS estimates (except that we set the Häagen-Dazs coefficient to zero) and the marginal costs that we have calculated to calculate the new prices `P_new`.

In [ ]:
P_new = []
s_new = []
for t in range(1,401,2):
  res = optimize.root(foc,[0,0],args=(C_hat[t-1:t+1],Xi_hat[t-1:t+1],[const,alpha,0]))
  print('Root found?',res.success)
  P_new = np.append(P_new,res.x)
  s_new = np.append(s_new,s(res.x,Xi_hat[t-1:t+1],[const,alpha,0]))

In [ ]:
print('The new Häagen-Dazs average unit price is $',round(P_new[0::2].mean(),2),', and the new average Ben & Jerry\'s unit price is $',round(P_new[1::2].mean(),2),'.')
print('The new average Häagen-Dazs market share is',round(100*s_new[0::2].mean(),2),'%. Ben & Jerry\'s average market share is',100*round(s_new[1::2].mean(),2),'%.')

We find that the prices and market shares of Häagen-Dazs is now on average larger than Ben & Jerry's. Can you explain why?

Next, we calculate Häagen-Dazs new (variable) profit.

In [ ]:
pi_1 = (data[data['Häagen-Dazs'] == 1]['county size']*s_new[1::2]*(P_new[1::2] - C_hat[1::2])).sum()
print('Häagen-Dazs\'s variable profit in the new equilibrium are $',pi_1.astype(int),'.')
print('Häagen-Dazs would benefit by $',(pi_1-pi_0).astype(int),' if its ice cream was as good as Ben \& Jerry\'s.')


## 8. RANDOM PRICE COEFFICIENT

First, we load a new dataset.

In [ ]:
product_data = pd.read_csv('demand-estimation/data2.csv',index_col=0)
product_data.head(5)

We want to estimate demand using the following utility specification:
$$ u_{ijt} = \beta_0 + \beta x_{jt} + \alpha_i p_{jt} + \xi_{jt} + \epsilon_{ijt}, \ \text{ where } \ \alpha_i\sim N(\alpha,\sigma^2)$$
To estimate the demand coefficients we use the `pyblp` package.

In [ ]:
!pip install pyblp
import pyblp

Next, we specify the model. We tell `pyblp` that $\beta_0,\beta,$ and $\alpha$ are the 'linear' coefficients ($X_2$).

In [ ]:
X1_formulation = pyblp.Formulation('1 + prices + x')

We specify that $\alpha_i$ is a **random coefficient**.

In [ ]:
X2_formulation = pyblp.Formulation('0 + prices')

We specify how `pyblp` integrates numerically and what algorithm to use to solve the problem.

In [ ]:
product_formulations = (X1_formulation, X2_formulation)
integration = pyblp.Integration('halton', size=1000, specification_options={'seed': 0})
bfgs = pyblp.Optimization('bfgs')
problem = pyblp.Problem(product_formulations, product_data, integration=integration)

We ask `pyblp` to estimate the model.

In [ ]:
results = problem.solve(sigma=1.0, optimization=bfgs)
beta0_hat,alpha_hat,beta1_hat = results.beta.flatten()
sigma_hat = results.sigma[0,0]
results

The estimates are pretty accurate. The true coefficients are:

\begin{align*}
  \beta_0 = & 2.6 \\
  \alpha = & -1.6 \\
  \beta_x = & -0.3 \\
  \sigma = & 0.25
\end{align*}

In [ ]:
ai = np.linspace(alpha_hat - 4*sigma_hat, alpha_hat + 4*sigma_hat, 1000)
pdf = norm.pdf(ai, alpha_hat, sigma_hat)
plt.figure(figsize=(8, 4.5))
plt.plot(ai, pdf, label=f'Normal PDF\nMean: {alpha_hat}, SD: {sigma_hat}')
plt.xlabel(r'$a_i$')
plt.ylabel('probability density')
plt.show()